##Import

In [0]:
!pip install sastrawi

     |████████████████████████████████| 215kB 3.5MB/s 


In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn import metrics

#SVD
from numpy import array
from numpy import diag
from numpy import dot
from numpy import zeros
from scipy.linalg import svd

##Read Data

In [0]:
#df_train = pd.read_csv("data/dataset.csv")
#df_train
from google.colab import drive
drive.mount('/content/drive/')

df_train = pd.read_csv("/content/drive/My Drive/DS/Project/datatest.csv")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


##Labeling

In [0]:
labelEncoder = LabelEncoder()
labelEncoder.fit(df_train['category'])
target = labelEncoder.transform(df_train['category'])
#target

##Preprocessing

In [0]:
#stopword_file = open("stopwords-id.txt", "r+")
stopword_file = open("/content/drive/My Drive/DS/Project/stopwords-id.txt", "r+")
stopwords = stopword_file.read()
stopword_file.close()
stopwords = stopwords.split("\n")
#stopwords

In [0]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

##TF-IDF

In [0]:
#stopword=open('https://raw.githubusercontent.com/stopwords-iso/stopwords-id/master/stopwords-id.txt',"r+")
vectorizer = TfidfVectorizer(stop_words= stopwords, preprocessor=stemmer.stem)
#vectorizer = TfidfVectorizer(stop_words= stopwords)
X = vectorizer.fit_transform(df_train['content'])
#X

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arti', 'asa', 'buah', 'bum', 'enak', 'hadap', 'ikan', 'ira', 'jenak', 'kali', 'kena', 'kerja', 'khusus', 'laku', 'langsung', 'lihat', 'maksud', 'masuk', 'mata', 'mena', 'nyata', 'olah', 'orang', 'rupa', 'sampa', 'sangkut', 'sekal', 'sekira', 'sin', 'tama', 'tuju', 'upa', 'utama'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
X = vectorizer.transform(df_train['content'])

In [0]:
#vectorizer.get_feature_names()

##SVD

In [0]:
svd = TruncatedSVD(n_components=3)
vector_reduced = svd.fit_transform(X)

##Clasification

In [0]:
classifier = SVC(kernel="rbf")
classifier.fit(X, target)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
MNB = MultinomialNB()
MNB.fit(X, target)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
GNB = GaussianNB()
GNB.fit(X.toarray(), target)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
BNB = BernoulliNB()
BNB.fit(X, target)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [0]:
from joblib import dump
dump(vectorizer, "tfidfvectorizer-prod.joblib")
dump(classifier, "svm-rbf-prod.joblib")
dump(MNB, "mnb-prod.joblib")
dump(GNB, "gnb-prod.joblib")
dump(BNB, "bnb-prod.joblib")
dump(labelEncoder, "labelEncoder.joblib")

['labelEncoder.joblib']

In [0]:
df_test = pd.read_csv("data/datatest.csv")
X = vectorizer.transform(df_test['content'])
Y_actual = labelEncoder.transform(df_test['category'])


In [0]:
print("SVM RBF Evaluation")
Y_pred = classifier.predict(X)
print(metrics.classification_report(Y_actual, Y_pred))

SVM RBF Evaluation
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       200
           1       1.00      1.00      1.00       200
           2       1.00      1.00      1.00       200
           3       1.00      0.99      1.00       200
           4       1.00      1.00      1.00       200

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [0]:
print("MNB Evaluation")
Y_pred = MNB.predict(X)
print(metrics.classification_report(Y_actual, Y_pred))

MNB Evaluation
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       200
           1       0.95      0.98      0.97       200
           2       0.99      0.99      0.99       200
           3       0.97      0.97      0.97       200
           4       0.99      0.93      0.96       200

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000



In [0]:
print("GNB Evaluation")
Y_pred = GNB.predict(X.toarray())
print(metrics.classification_report(Y_actual, Y_pred))

GNB Evaluation
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       200
           1       1.00      1.00      1.00       200
           2       1.00      1.00      1.00       200
           3       1.00      1.00      1.00       200
           4       1.00      1.00      1.00       200

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [0]:
print("BNB Evaluation")
Y_pred = BNB.predict(X)
print(metrics.classification_report(Y_actual, Y_pred))

BNB Evaluation
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       200
           1       0.98      0.96      0.97       200
           2       1.00      1.00      1.00       200
           3       0.97      1.00      0.98       200
           4       0.98      0.96      0.97       200

    accuracy                           0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000

